In [2]:
!pip install dask[complete]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 98.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [distributed]


In [3]:
import numpy as np
import pandas as pd

In [5]:
n = 5_000_000
df = pd.DataFrame({
    "id":np.arange(n),
    "value":np.random.normal(100,20,size=n),
    "category":np.random.choice(['A','B','C'],size=n)
})

df.to_csv('big_data.csv',index=False)

In [6]:
#czytanie big data
import dask.dataframe as dd

ddf = dd.read_csv("big_data.csv")
print(ddf.head(10))

   id       value category
0   0  114.839376        B
1   1   95.225090        A
2   2  126.299426        C
3   3   98.058411        B
4   4   78.025873        C
5   5  121.681937        C
6   6   62.890475        C
7   7   97.800452        C
8   8   97.128029        B
9   9  116.221999        C


In [8]:
#proste operacje
mean_val = ddf["value"].mean()
print(f"Średnia  (leniwa): {mean_val}")
print(f"Średnia (obliczona): {mean_val.compute()}")

Średnia  (leniwa): <dask_expr.expr.Scalar: expr=ArrowStringConversion(frame=FromMapProjectable(33a2dc7))['value'].mean(), dtype=float64>
Średnia (obliczona): 99.9988998165767


In [9]:
grouped = ddf.groupby("category")["value"].mean()
print(f"średnia per kategoria (leniwa)): {grouped}")
print(f"średnia per kategoria (obliczona)): {grouped.compute()}")

średnia per kategoria (leniwa)): Dask Series Structure:
npartitions=1
    float64
        ...
Dask Name: getitem, 5 expressions
Expr=((ArrowStringConversion(frame=FromMapProjectable(33a2dc7))[['value', 'category']]).mean(observed=False, chunk_kwargs={'numeric_only': False}, aggregate_kwargs={'numeric_only': False}, _slice='value'))['value']
średnia per kategoria (obliczona)): category
A    100.000226
B     99.998169
C     99.998303
Name: value, dtype: float64


In [10]:
print(f"liczba partycji: {ddf.npartitions}")
print(f"rozmiar danych szacowaany {ddf.memory_usage(deep=True).compute()}")

liczba partycji: 2
rozmiar danych szacowaany Index            264
id          40000000
value       40000000
category    45000000
dtype: int64
